### Import packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from functools import reduce

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import imblearn.over_sampling
from sklearn.metrics import confusion_matrix

/Users/courtney/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/courtney/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


### Import data 

In [4]:
outcome_df = pd.read_csv('aac_intakes_outcomes.csv')
intake_df = pd.read_csv('aac_intakes.csv')
popularity_df = pd.read_csv('akc_popular_breeds_2013-2016.csv')
# if time: scrape 2017 popularity data
intelligence_df = pd.read_csv('dog_intelligence.csv')
# best_in_show_df = pd.read_csv('best_in_show_westminster.csv')
# if time: get data on cost, other stuff
# if time: group membership of dog
# breed_size = pd.read_csv('akc_breed_info.csv')

### Inital exploration of features 

#### Intakes dataframe 

In [5]:
intake_df.columns

Index(['age_upon_intake', 'animal_id', 'animal_type', 'breed', 'color',
       'datetime', 'datetime2', 'found_location', 'intake_condition',
       'intake_type', 'name', 'sex_upon_intake'],
      dtype='object')

In [6]:
intake_df.name.isnull().sum()

24584

In [7]:
intake_df['has_name'] = np.where(intake_df.name.isnull() == True, 0, 1)
intake_df.has_name.value_counts()

1    55603
0    24584
Name: has_name, dtype: int64

#### Intakes and Outcomes dataframe 

In [8]:
outcome_df.columns

Index(['age_upon_outcome', 'animal_id_outcome', 'date_of_birth',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome',
       'age_upon_outcome_(days)', 'age_upon_outcome_(years)',
       'age_upon_outcome_age_group', 'outcome_datetime', 'outcome_month',
       'outcome_year', 'outcome_monthyear', 'outcome_weekday', 'outcome_hour',
       'outcome_number', 'dob_year', 'dob_month', 'dob_monthyear',
       'age_upon_intake', 'animal_id_intake', 'animal_type', 'breed', 'color',
       'found_location', 'intake_condition', 'intake_type', 'sex_upon_intake',
       'count', 'age_upon_intake_(days)', 'age_upon_intake_(years)',
       'age_upon_intake_age_group', 'intake_datetime', 'intake_month',
       'intake_year', 'intake_monthyear', 'intake_weekday', 'intake_hour',
       'intake_number', 'time_in_shelter', 'time_in_shelter_days'],
      dtype='object')

In [13]:
yearly_animals = outcome_df.groupby(by=['animal_type', 'intake_year']).size().reset_index()
yearly_mean = yearly_animals.groupby(by='animal_type').mean()

In [14]:
print(outcome_df.dob_year.min())
print(outcome_df.dob_year.max())

1991
2018


In [15]:
outcome_df.intake_year.value_counts()

2015    18699
2014    18645
2016    17632
2017    17440
2013     4178
2018     3078
Name: intake_year, dtype: int64

In [16]:
outcome_df.outcome_type.value_counts()

Adoption           33594
Transfer           23799
Return to Owner    14791
Euthanasia          6244
Died                 690
Disposal             304
Rto-Adopt            179
Missing               46
Relocate              15
Name: outcome_type, dtype: int64

In [17]:
outcome_df.animal_type.value_counts()

Dog      45366
Cat      29539
Other     4428
Bird       339
Name: animal_type, dtype: int64

In [18]:
outcome_df.intake_type.value_counts()

Stray                 55935
Owner Surrender       15028
Public Assist          4994
Wildlife               3464
Euthanasia Request      251
Name: intake_type, dtype: int64

In [19]:
outcome_df.breed.value_counts()

Domestic Shorthair Mix                            23423
Pit Bull Mix                                       6256
Chihuahua Shorthair Mix                            4831
Labrador Retriever Mix                             4789
Domestic Medium Hair Mix                           2326
German Shepherd Mix                                1950
Bat Mix                                            1381
Domestic Longhair Mix                              1248
Australian Cattle Dog Mix                          1099
Siamese Mix                                         996
Bat                                                 827
Dachshund Mix                                       811
Boxer Mix                                           683
Miniature Poodle Mix                                662
Border Collie Mix                                   662
Catahoula Mix                                       480
Raccoon Mix                                         471
Rat Terrier Mix                                 

### Clean intake and outcome data

In [20]:
names = pd.concat([intake_df.animal_id, intake_df.has_name], axis=1)
names.columns = ['animal_id_intake', 'has_name']

In [21]:
outcome_df.columns = ['age_upon_outcome', 'animal_id_outcome', 'date_of_birth',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome',
       'age_upon_outcome_days', 'age_upon_outcome_years',
       'age_upon_outcome_age_group', 'outcome_datetime', 'outcome_month',
       'outcome_year', 'outcome_monthyear', 'outcome_weekday', 'outcome_hour',
       'outcome_number', 'dob_year', 'dob_month', 'dob_monthyear',
       'age_upon_intake', 'animal_id_intake', 'animal_type', 'breed', 'color',
       'found_location', 'intake_condition', 'intake_type', 'sex_upon_intake',
       'count', 'age_upon_intake_days', 'age_upon_intake_years',
       'age_upon_intake_age_group', 'intake_datetime', 'intake_month',
       'intake_year', 'intake_monthyear', 'intake_weekday', 'intake_hour',
       'intake_number', 'time_in_shelter', 'time_in_shelter_days']

In [22]:
print(outcome_df.color.value_counts().head())
threshold = 1000
outcome_df['adjusted_color'] = outcome_df.color
color_freq = outcome_df.color.value_counts()
to_remove = color_freq[color_freq <= threshold].index
outcome_df.adjusted_color.replace(to_remove, 'Other_color', inplace=True)

Black/White    8270
Black          6673
Brown Tabby    4471
Brown          3598
White          2835
Name: color, dtype: int64


In [23]:
print(outcome_df.breed.value_counts().head())
threshold = 100
outcome_df['adjusted_breed'] = outcome_df.breed
breed_freq = outcome_df.breed.value_counts()
to_remove = breed_freq[breed_freq <= threshold].index
outcome_df.adjusted_breed.replace(to_remove, 'Other_breed', inplace=True)

Domestic Shorthair Mix      23423
Pit Bull Mix                 6256
Chihuahua Shorthair Mix      4831
Labrador Retriever Mix       4789
Domestic Medium Hair Mix     2326
Name: breed, dtype: int64


In [24]:
print(outcome_df.shape)
outcome_df.head()

(79672, 43)


,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_days,age_upon_outcome_years,age_upon_outcome_age_group,outcome_datetime,...,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,adjusted_color,adjusted_breed
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,Other_color,Other_breed
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,Other_color,Other_breed
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,Other_color,Other_breed
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,Tricolor,Dachshund
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,Brown/White,Other_breed


In [25]:
names.animal_id_intake.value_counts().head()

A721033    13
A718223    11
A706536    11
A716018     9
A616444     8
Name: animal_id_intake, dtype: int64

In [26]:
outcome_df.shape

(79672, 43)

In [27]:
outcome_df.animal_id_intake.head()

0    A006100
1    A006100
2    A006100
3    A047759
4    A134067
Name: animal_id_intake, dtype: object

In [28]:
outcome_df = outcome_df.merge(names.drop_duplicates(), on='animal_id_intake', how='inner')

In [29]:
outcome_df.shape

(79672, 44)

In [30]:
mask = (outcome_df.outcome_type.isin(['Adoption', 'Transfer', 'Return to Owner', 'Euthanasia']))
df_main_outcomes = outcome_df[mask]

In [31]:
df_main_outcomes.shape

(78428, 44)

In [32]:
species_mask = df_main_outcomes.animal_type.isin(['Dog', 'Cat'])
df_catdog = df_main_outcomes[species_mask]

In [33]:
df_catdog.shape

(74109, 44)

In [34]:
df_catdog.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_days,age_upon_outcome_years,age_upon_outcome_age_group,outcome_datetime,...,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,adjusted_color,adjusted_breed,has_name
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,Other_color,Other_breed,1
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,Other_color,Other_breed,1
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,Other_color,Other_breed,1
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,Tricolor,Dachshund,1
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,Brown/White,Other_breed,1


In [35]:
print(df_catdog.intake_condition.value_counts())
intake_status = pd.DataFrame(np.where(df_catdog.intake_condition == 'Normal', 0, 1))
intake_status.columns = ['intake_sick']
intake_status.shape

Normal      66454
Injured      3185
Sick         2090
Nursing      1793
Aged          309
Other         141
Feral          91
Pregnant       46
Name: intake_condition, dtype: int64


(74109, 1)

In [36]:
intake_sex = pd.get_dummies(df_catdog.sex_upon_intake)
intake_sex.columns = ['intact_female', 'intact_male', 'neutered_male', 'spayed_female', 'sex_unknown']

In [37]:
is_intact = pd.DataFrame(np.where(df_catdog.sex_upon_intake.str.contains("Intact"), 1, 0))
is_intact.columns = ['is_intact']

In [38]:
is_intact.shape

(74109, 1)

In [39]:
happy = pd.DataFrame(np.where(df_catdog.outcome_type.isin(['Adoption', 'Return to Owner']), 1, 0))
happy.columns = ['happy_outcome']

In [40]:
happy.shape

(74109, 1)

In [41]:
outcome_classes = pd.factorize(df_catdog.outcome_type)
outcome = pd.DataFrame(list(outcome_classes[0]))
outcome.columns = ['outcome_class']

In [42]:
is_dog = pd.DataFrame(np.where(df_catdog.animal_type == 'Dog', 1, 0))
is_dog.columns = ['dog']

In [43]:
is_dog.shape

(74109, 1)

In [44]:
print(df_catdog.intake_type.value_counts())
intake_type = pd.get_dummies(df_catdog.intake_type)
intake_type.columns = ['euthanasia_request', 'owner_surrender', 'public_assist', 'stray']

Stray                 54585
Owner Surrender       14544
Public Assist          4743
Euthanasia Request      237
Name: intake_type, dtype: int64


In [45]:
intake_type.shape

(74109, 4)

In [46]:
intake_date = df_catdog[['intake_month', 'intake_year', 'intake_hour']]
intake_weekday = pd.get_dummies(df_catdog.intake_weekday)

In [47]:
intake_date.shape

(74109, 3)

In [48]:
intake_weekday.shape

(74109, 7)

In [49]:
colors = pd.get_dummies(df_catdog.adjusted_color)
colors.shape

(74109, 26)

In [50]:
breeds = pd.get_dummies(df_catdog.adjusted_breed)
breeds.shape

(74109, 72)

In [51]:
weekend_intake = pd.DataFrame(np.where(df_catdog.intake_weekday.isin(['Saturday', 'Sunday']), 1, 0))
weekend_intake.columns = ['intake_is_weekend']
weekend_intake.shape

(74109, 1)

In [52]:
df = intake_status.join([intake_sex, is_intact, is_dog, intake_type, intake_date, intake_weekday, weekend_intake, colors, breeds, happy, outcome])
df.shape

(74109, 123)

### Add in untransformed features 

In [53]:
df['has_name'] = df_catdog.has_name
df['age_days'] = df_catdog.age_upon_intake_days

In [54]:
df.shape

(74109, 125)

In [55]:
df.head()

,intake_sick,intact_female,intact_male,neutered_male,spayed_female,sex_unknown,is_intact,dog,euthanasia_request,owner_surrender,...,Siberian Husky,Siberian Husky Mix,Snowshoe Mix,Staffordshire Mix,Toy Poodle Mix,Yorkshire Terrier Mix,happy_outcome,outcome_class,has_name,age_days
0,0,0.0,0.0,1.0,0.0,0.0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,3650.0
1,0,0.0,0.0,1.0,0.0,0.0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,2555.0
2,0,0.0,0.0,1.0,0.0,0.0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,2190.0
3,0,0.0,0.0,1.0,0.0,0.0,0,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1.0,3650.0
4,1,0.0,0.0,1.0,0.0,0.0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,5840.0


In [56]:
pickle.dump(df, open("clean_data.p", "wb"))